![spotify_logo](../img/spotify_logo.png)

# Spotify REST API Challenge

__What to listen?__

Create your own playlist based on the related artists and their most popular tracks using the [Spotify REST API](https://developer.spotify.com/)

In [1]:
# Imports
import requests
import pandas as pd 



### Get access!!!

Get your `client_id` and `client_secret` to generate your __token__ access

In [2]:
# Generate token with a POST request

client_id = "7ab93763586c4576824105536fdfe118"# CLIENT ID
client_secret = "1b8b29e455074135b46061928c4c36e2"# CLIENT SECRET
auth_url = 'https://accounts.spotify.com/api/token'

In [3]:
auth_response = requests.post(auth_url, {'grant_type': 'client_credentials',
                                         'client_id': client_id,
                                         'client_secret': client_secret}).json()

In [4]:
access_token = auth_response['access_token']
auth_response

{'access_token': 'BQDUg5DTD-Tu1EeewlrCxSoHs89tNAaQWG4qOa2ZaesrDAMQG1FdYAXOTNqV80YEtqFLB8aYh8BEUqoPY1jYVylvnNP203PcBi8GCLJGetTCcJqzFDw',
 'token_type': 'Bearer',
 'expires_in': 3600}

### Set your main variables!!!

Set the `base_uri` (i.e.: end-point), parameters and `headers` for your __GET__ operations

In [5]:
# Base end-point construction

base_url = 'https://api.spotify.com/v1/'
resource = 'artists/'

header_info = {'Authorization': 'Bearer {token}'.format(token=access_token)}

### Create your new playlist!!!

Use [`/related-artists`](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-an-artists-related-artists) and [`/top-tracks`](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-an-artists-top-tracks) in order to get the data that you need.

__Here you have the different artists to start with:__

- __Option 1:__ 0L8ExT028jH3ddEcZwqJJ5

- __Option 2:__ 4Y7tXHSEejGu1vQ9bwDdXW

- __Option 3:__ 6FBDaR13swtiWwGhX1WQsP

- __Option 4:__ 0kyQwKHCZnKE7kTXkxXjrB

- __Option 5:__ 2d0hyoQ5ynDBnkvAbJKORj

- __Option 6:__ 3bgsNtcf5d5h9jbQbohfBK

- __Option 7:__ 7mWCSSOYqm4E9mB7V4ot6S

- __Option 8:__ 64KEffDW9EtZ1y2vBYgq8T

- __Option 9:__ 4k1ELeJKT1ISyDv8JivPpB

- __Option 10:__ 4Z8W4fKeB5YxbusRsdQVPb

- __Option 11:__ 26dSoYclwsYLMAKD3tpOr4

- __Option 12:__ 7y97mc3bZRFXzT2szRM4L4

- __Option 13:__ 1w5Kfo2jwwIPruYS2UWh56


> Remember to check the [Requests](https://requests.readthedocs.io/en/latest/) library docs!!!

---

In [6]:
# Here you can complete your endpoint URI
artist_id = "0WwSkZ7LtFUFjGjMZBMt6T"
method = "/related-artists"
full_endpoint = base_url + resource + artist_id + method
full_endpoint

'https://api.spotify.com/v1/artists/0WwSkZ7LtFUFjGjMZBMt6T/related-artists'

In [7]:
# Start building your playlist!!!
related_artist_response = requests.get(full_endpoint, headers = header_info)
related_artist_response.status_code


200

In [8]:
related_artist_response.content

b'{\n  "artists" : [ {\n    "external_urls" : {\n      "spotify" : "https://open.spotify.com/artist/0FI0kxP0BWurTz8cB8BBug"\n    },\n    "followers" : {\n      "href" : null,\n      "total" : 1477538\n    },\n    "genres" : [ "soft rock" ],\n    "href" : "https://api.spotify.com/v1/artists/0FI0kxP0BWurTz8cB8BBug",\n    "id" : "0FI0kxP0BWurTz8cB8BBug",\n    "images" : [ {\n      "height" : 640,\n      "url" : "https://i.scdn.co/image/ab6761610000e5eb685c708602ec164b04e38e26",\n      "width" : 640\n    }, {\n      "height" : 320,\n      "url" : "https://i.scdn.co/image/ab67616100005174685c708602ec164b04e38e26",\n      "width" : 320\n    }, {\n      "height" : 160,\n      "url" : "https://i.scdn.co/image/ab6761610000f178685c708602ec164b04e38e26",\n      "width" : 160\n    } ],\n    "name" : "Mark Knopfler",\n    "popularity" : 58,\n    "type" : "artist",\n    "uri" : "spotify:artist:0FI0kxP0BWurTz8cB8BBug"\n  }, {\n    "external_urls" : {\n      "spotify" : "https://open.spotify.com/artis

In [9]:
related_artists_json = related_artist_response.json()

In [10]:
[related_artists_json["artists"][i]["name"] for i in range(len(related_artists_json["artists"]))]

['Mark Knopfler',
 'Supertramp',
 'Joe Cocker',
 'Chris Rea',
 'The Police',
 'Genesis',
 'Eric Clapton',
 'Bruce Springsteen',
 'R.E.M.',
 'Deep Purple',
 'The Alan Parsons Project',
 'Simple Minds',
 'David Gilmour',
 'Peter Gabriel',
 'Jethro Tull',
 'Pink Floyd',
 'U2',
 'The Who',
 'Phil Collins',
 'ZZ Top']

In [11]:
artist_id = "0WwSkZ7LtFUFjGjMZBMt6T"
method = "/top-tracks"
argument = "?market=ES"
full_endpoint_tracks = base_url + resource + artist_id + method + argument
full_endpoint_tracks

'https://api.spotify.com/v1/artists/0WwSkZ7LtFUFjGjMZBMt6T/top-tracks?market=ES'

In [12]:
related_artist_response_tracks = requests.get(full_endpoint_tracks, headers = header_info)
related_artist_response_tracks.status_code

200

In [13]:
related_artist_response_tracks_json = related_artist_response_tracks.json()
related_artist_response_tracks_json["tracks"][0]["name"]

'Sultans Of Swing'

In [14]:
related_artists_id = [related_artists_json["artists"][i]["id"] for i in range(len(related_artists_json["artists"]))]
related_artists_id

['0FI0kxP0BWurTz8cB8BBug',
 '3JsMj0DEzyWc0VDlHuy9Bx',
 '3pFCERyEiP5xeN2EsPXhjI',
 '5KEG7G8LDYlHgFDqZyEEs2',
 '5NGO30tJxFlKixkPSgXcFE',
 '3CkvROUTQ6nRi9yQOcsB50',
 '6PAt558ZEZl0DmdXlnjMgD',
 '3eqjTLE0HfPfh78zjh6TqT',
 '4KWTAlx2RvbpseOGMEmROg',
 '568ZhdwyaiCyOGJRtNYhWf',
 '2m62cc253Xvd9qYQ8d2X3d',
 '6hN9F0iuULZYWXppob22Aj',
 '2FcC4sDMXme2ziI7tGKMK8',
 '7C4sUpWGlTy7IANjruj02I',
 '6w6z8m4WXX7Tub4Rb6Lu7R',
 '0k17h0D3J5VfsdmQ1iZtE9',
 '51Blml2LZPmy7TTiAg47vQ',
 '67ea9eGLXYMsO2eYQRui3w',
 '4lxfqrEsLX6N1N4OCSkILp',
 '2AM4ilv6UzW0uMRuqKtDgN']

#method = "/top-tracks"
#argument = "?market=ES"

#for i in related_artists_id:
    #print(base_url + resource + i + method + argument)



#related_artist_top_tracks = [requests.get(base_url + resource + i + method + argument, headers = header_info) for i in related_artists_id]
#related_artist_top_tracks[1].status_code

#related_artist_top_tracks_json = [related_artist_top_tracks[x].json()for x in range(len(related_artist_top_tracks))]
#related_artist_top_tracks_json[1]
#related_artist_top_tracks[1].status_code

#for i in related_artist_top_tracks_json:
   # for x in i:
   #     print (x)

In [36]:
def get_top_tracks(id_artist):
    resource = "/artists"
    parameters = f"/{id_artist}/top-tracks?market=ES"
    url = base_url + resource + parameters
    response = requests.get(url, headers = header_info).json()
    df_tracks = pd.DataFrame(response["tracks"])[["id", "href", "name", "uri"]]
    df_tracks["related_id"]= id_artist
    return df_tracks

In [37]:
get_top_tracks(id_artist ='67ea9eGLXYMsO2eYQRui3w')

KeyError: 'tracks'

In [48]:
id_selected ='568ZhdwyaiCyOGJRtNYhWf'

In [49]:
resource = "artists"
parameters = f'/{id_selected}'
url = base_url + resource + parameters
print (url)

https://api.spotify.com/v1/artists/568ZhdwyaiCyOGJRtNYhWf


In [50]:
response = requests.get(url, headers = header_info).json()

In [51]:
response

{'external_urls': {'spotify': 'https://open.spotify.com/artist/568ZhdwyaiCyOGJRtNYhWf'},
 'followers': {'href': None, 'total': 5238333},
 'genres': ['album rock', 'classic rock', 'hard rock', 'metal', 'rock'],
 'href': 'https://api.spotify.com/v1/artists/568ZhdwyaiCyOGJRtNYhWf',
 'id': '568ZhdwyaiCyOGJRtNYhWf',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb607e7523c28dd40288a08df4',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab67616100005174607e7523c28dd40288a08df4',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f178607e7523c28dd40288a08df4',
   'width': 160}],
 'name': 'Deep Purple',
 'popularity': 63,
 'type': 'artist',
 'uri': 'spotify:artist:568ZhdwyaiCyOGJRtNYhWf'}

In [52]:
dict_artist = dict((key, [response[key]])for key in ["id", "name", "genres", "popularity","followers"] if key in response)

In [53]:
df_artist = pd.DataFrame(dict_artist)
df_artist

,id,name,genres,popularity,followers
0,568ZhdwyaiCyOGJRtNYhWf,Deep Purple,"[album rock, classic rock, hard rock, metal, r...",63,"{'href': None, 'total': 5238333}"


In [54]:
df_artist = df_artist.rename(columns = {"id" : "artist_id",
                                        "name" : "artist_name",
                                       "genres" : "artist_genres",
                                       "popularity": "artist_popularity",
                                       "followers": "artist_followers"})

In [55]:
df_artist

,artist_id,artist_name,artist_genres,artist_popularity,artist_followers
0,568ZhdwyaiCyOGJRtNYhWf,Deep Purple,"[album rock, classic rock, hard rock, metal, r...",63,"{'href': None, 'total': 5238333}"


In [56]:
df_artist["followers_clean"] = df_artist["artist_followers"].apply(lambda x: x["total"] if "total" in x else None)
df_artist

,artist_id,artist_name,artist_genres,artist_popularity,artist_followers,followers_clean
0,568ZhdwyaiCyOGJRtNYhWf,Deep Purple,"[album rock, classic rock, hard rock, metal, r...",63,"{'href': None, 'total': 5238333}",5238333


In [60]:
id_related = "51Blml2LZPmy7TTiAg47vQ"
resource = "artists"
parameters_related = f"/{id_related}/related-artists"
url_related = base_url + resource + parameters_related
print (url_related)

https://api.spotify.com/v1/artists/51Blml2LZPmy7TTiAg47vQ/related-artists


In [64]:
response = requests.get(url_related, headers = header_info).json()
df_related_artist = pd.DataFrame(response ["artists"])[["id","name","genres","popularity","followers"]]
df_related_artist = df_related_artist.rename(columns = {"id" : "related_id",
                                        "name" : "related_name",
                                       "genres" : "related_genres",
                                       "popularity": "related_popularity",
                                       "followers": "related_followers"})
df_related_artist["artist_id"]= id_selected
df_related_artists = pd.merge (df_related_artist, df_artist, on = "artist_id")
df_related_artists.head()

,related_id,related_name,related_genres,related_popularity,related_followers,artist_id,artist_name,artist_genres,artist_popularity,artist_followers,followers_clean
0,4KWTAlx2RvbpseOGMEmROg,R.E.M.,"[alternative rock, athens indie, permanent wav...",71,"{'href': None, 'total': 4233689}",568ZhdwyaiCyOGJRtNYhWf,Deep Purple,"[album rock, classic rock, hard rock, metal, r...",63,"{'href': None, 'total': 5238333}",5238333
1,1eClJfHLoDI4rZe5HxzBFv,INXS,"[australian rock, dance rock, funk rock, new r...",68,"{'href': None, 'total': 2499365}",568ZhdwyaiCyOGJRtNYhWf,Deep Purple,"[album rock, classic rock, hard rock, metal, r...",63,"{'href': None, 'total': 5238333}",5238333
2,5NGO30tJxFlKixkPSgXcFE,The Police,"[album rock, classic rock, permanent wave, rock]",74,"{'href': None, 'total': 5697330}",568ZhdwyaiCyOGJRtNYhWf,Deep Purple,"[album rock, classic rock, hard rock, metal, r...",63,"{'href': None, 'total': 5238333}",5238333
3,6hN9F0iuULZYWXppob22Aj,Simple Minds,"[album rock, dance rock, new romantic, new wav...",64,"{'href': None, 'total': 1883060}",568ZhdwyaiCyOGJRtNYhWf,Deep Purple,"[album rock, classic rock, hard rock, metal, r...",63,"{'href': None, 'total': 5238333}",5238333
4,5gznATMVO85ZcLTkE9ULU7,Lenny Kravitz,"[permanent wave, rock]",67,"{'href': None, 'total': 3048167}",568ZhdwyaiCyOGJRtNYhWf,Deep Purple,"[album rock, classic rock, hard rock, metal, r...",63,"{'href': None, 'total': 5238333}",5238333


---

### Bonus track!!!

You can publish your own Playlist with [`/playlists`](https://developer.spotify.com/documentation/web-api/reference/#/operations/create-playlist) and [`/tracks`](https://developer.spotify.com/documentation/web-api/reference/#/operations/add-tracks-to-playlist).

![I quit](https://media.giphy.com/media/gui67fZ3xIneM/giphy.gif)

In [ ]:
# Bonus
from dotenv import dotenv_value
config = dotenv_values (".env")
print(config)


In [ ]:
client_id = config["client_ID"]
client_secret = config["client_secret"]

In [ ]:
auth_response = requests.post(auth_url{"grant_type": "client_credentials"})

In [ ]:
import base64
user_id = 
auth_header = base64.b64encode(f'{client_id}:{client_secret}'. encode("utf-8")).decode("utf-8")
auth_options = {
    "url":"https://accounts.spotify.com/api/token"
    , "headers":{
        "Authorization" : f'Basic {auth_header}'
    }, "data": {
        "grant_type": "client_credentials"
    }
}
response = requests.post(**auth_options)
if response.status_code == 200: 
    token = response.json(). get ("access_token")
    api_url

In [ ]:
import webbrowser
from urllib.pase import urlparse, parse_qs
client_id = client_id
redirect_uri = #copy url from app
scopes = "playlist-modify-public" #type of list to create, public or private
authorization_url = #look in documentation! https://accounts.spotify.com/authorize?client_id=%7Bclient_id%7D&response_type=token&redirect_uri=%7Bredirect_uri%7D&scope=%7Bscopes%7D
webbrowser.open(authorization_url)

In [ ]:
def get_access_token(): 
    auth_url = input("Enter the url after being directed: ")
    parsed_url = urlparse

In [ ]:
url = 'https://example.com/callback?code=https://example.com/callback?code=AQAVvyDgpT2w1GwMABedjC75Sd2UJs3bCZRe6SjKuiLfgHtq9SQ6mpv-5Hqb9uL4QskKccuBQozqoLWc9HhYhgOzPsi_3pN2cpJFpFxNt39JVsOPHQg-98gGhgqgmZChtBsJuBqVES1JPYW9eYUy5UoZ8j-ev0VI5l8k-o-uGqk9ABbpjSsyrM-RhLwNis1pinDMKHFuC6sJGhyDcR-iwUN1AOcuTdm2C1NBsqAdW2TTHGge68gvt_VqmHnZCPn5vcCurWXhLot4NQ9xqBDNIFXXtV5nR78rYFD95DSYwTh6ePNxPw&state=gGqVja50MMklpaHc-_eK1UtjXPJyeQxjl_sQ-SwTZaF0txfqBaQ&state=bSCPsLTi7HItyicy'
access_token = url.replace('https://example.com/callback?code=','').split('&state')[0]

In [ ]:
# Replace these variables with your own values
#access_token = “your_access_token_here”
user_id = “1147225367" # “your_user_id_here”
playlist_name = input(“Enter a playlist name:“)
playlist_description = f”Description of your playlist {playlist_name}”
# Define the API endpoint and headers
api_endpoint = f”https://api.spotify.com/v1/users/{user_id}/playlists”
headers = {
    “Authorization”: f”Bearer {access_token}“,
    “Content-Type”: “application/json”
}
# Create the playlist
playlist_data = {
    “name”: playlist_name,
    “description”: playlist_description,
    “public”: True  # You can change the visibility as needed
}
response = requests.post(api_endpoint, headers=headers, json=playlist_data)
if response.status_code == 201:
    playlist_info = response.json()
    print(“Playlist created successfully:“)
    print(playlist_info)
else:
    print(“Failed to create the playlist.“)

In [ ]:
def add_tracks_to_playlist(access_token, playlist_id, track_uris):
    api_endpoint = f”https://api.spotify.com/v1/playlists/{playlist_id}/tracks”
    headers = {
        “Authorization”: f”Bearer {access_token}“,
        “Content-Type”: “application/json”
    }
    data = {
        “uris”: track_uris
    }
    response = requests.post(api_endpoint, headers=headers, json=data)
    if response.status_code == 201:
        print(“Tracks added to the playlist successfully.“)
    else:
        print(“Failed to add tracks to the playlist")

---

You can always try with the [wrapper](https://github.com/plamere/spotipy)!!!

![I quit](https://media.giphy.com/media/3oFzlXPvXYZ4q8VQOs/giphy.gif)

---